In [ ]:
# pip install opencv-python

import cv2, urllib, datetime, numpy as np 
from IPython.display import clear_output, display
from PIL import Image

def scale(*t, d='UP'): 
    if d=='UP':
        c = 2
    else: 
        c = 0.5
    return *[int(i * c) for i in t],

mjpg_url = 

stream = urllib.request.urlopen(mjpg_url) 
#stream = urllib.request.urlopen() 

bytes, firstFrame, CTN = b'', None, True 
while CTN: 
    bytes += stream.read(1024)
    a = bytes.find(b'\xff\xd8')
    b = bytes.find(b'\xff\xd9') 
    if a != -1 and b != -1: 
        jpg = bytes[a:b+2]
        bytes = bytes[b+2:] 
        img = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_UNCHANGED) # 480, 640 
        img = img[30:, 80:300] # height width  # Only care about a portion 
        height, width, depth = img.shape
        bgr = cv2.resize(img, scale(width, height, d='Down'))# (width, height)  
        grey = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY) #  COLOR_BGR2RGB  
        blur = cv2.GaussianBlur(grey, (21, 21), 0) 
        if firstFrame is None: firstFrame = blur 
        frameDelta = cv2.absdiff(firstFrame, blur)  
        thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1] 
        thresh = cv2.dilate(thresh, None, iterations=2)
        cnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
        for c in cnts:  
            if cv2.contourArea(c) > 300:  # if the contour is too small, ignore it  
                (x, y, w, h) = cv2.boundingRect(c)                          # compute the bounding box for the contour
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #  COLOR_BGR2RGB
                cv2.rectangle(img, scale(x, y), scale(x + w, y + h), (0, 255, 0), 2) # draw the bounding box on original frame 
                cv2.putText(img, datetime.datetime.now().strftime("%I:%M:%S%p"), 
                            scale(x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                clear_output(wait=True)  
                display(Image.fromarray(img))  
                break